In [21]:
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

In [22]:
#Load the standard file
pickle_file = 'aflw_dataset.pickle'
with open(pickle_file,'rb') as f:
    save = pickle.load(f)
    print(save)
    training_dataset = save['training_dataset']
    training_label = save['training_label']
#     validation_dataset = save['validation_dataset']
#     validation_label = save['validation_label']
    test_dataset = save['test_dataset']
    test_label = save['test_label']
    del save  # hint to help gc free up memory
    print('Training set', training_dataset.shape, training_label.shape)
#     print('Validation set', validation_dataset.shape, validation_label.shape)
    print('Test set', test_dataset.shape, test_label.shape)
    #Normalising the images
    training_dataset -= 127
#     validation_dataset -= 127
    test_dataset -= 127

{'training_dataset': array([[ 0.8425197 ,  0.6614173 ,  0.6692913 , ..., -0.96850395,
        -0.96062994, -1.007874  ],
       [ 0.8976378 ,  0.9133858 ,  0.92913383, ..., -0.36220473,
        -0.36220473, -0.36220473],
       [ 0.78740156,  0.77952754,  0.7637795 , ..., -0.9212598 ,
        -0.9133858 , -0.8897638 ],
       ...,
       [ 0.8110236 ,  0.86614174,  0.81889766, ..., -0.56692916,
        -0.5590551 , -0.5590551 ],
       [ 0.9527559 , -0.8267717 ,  1.        , ...,  0.8503937 ,
         0.8897638 ,  0.9133858 ],
       [ 0.37795275,  0.37007874,  0.39370078, ...,  0.6692913 ,
         0.68503934, -0.97637796]], dtype=float32), 'training_label': array([[ 38.197186,  28.647888],
       [-19.098593, -28.647888],
       [-38.197186, -38.197186],
       ...,
       [  9.549296, -57.295776],
       [-38.197186,   9.549296],
       [  9.549296,   0.      ]], dtype=float32), 'test_dataset': array([[ 0.8110236 ,  0.86614174,  0.8897638 , ..., -0.43307087,
        -0.43307087, -0.

In [3]:
def final_test(predictions, labels):

    row, col = labels.shape

#     roll_prediction = np.copy(predictions[:,0])
    pitch_prediction = np.copy(predictions[:,0])
    yaw_prediction = np.copy(predictions[:,1])
#     roll_labels = np.copy(labels[:,0])
    pitch_labels = np.copy(labels[:,0])
    yaw_labels = np.copy(labels[:,1])

    #To degree
#     roll_prediction *= 25 # to degree
    pitch_prediction *= 45 # to degree
    yaw_prediction *= 90 # to degree

    #Root mean square error
#     roll_mean_error = np.sum(np.square(roll_prediction - roll_labels)) * 1/row
#     roll_mean_error = np.sqrt(roll_mean_error)
    pitch_mean_error = np.sum(np.square(pitch_prediction - pitch_labels)) * 1/row
    pitch_mean_error = np.sqrt(pitch_mean_error)
    yaw_mean_error = np.sum(np.square(yaw_prediction - yaw_labels)) * 1/row
    yaw_mean_error = np.sqrt(yaw_mean_error)

    print("=== TEST MEAN ERROR (DEGREE) ===")
#     print("ROLL: " + str(roll_mean_error))
    print("PITCH: " + str(pitch_mean_error))
    print("YAW: " + str(yaw_mean_error))


In [14]:
def accuracy(predictions, labels, verbose=False):
    #Pitch
    N, col = labels.shape
    if(verbose == True):
        print("PRED:   " + str(predictions[0,:]))
        print("LABEL:  " + str(labels[0,:]))

        #First value
        pred_value = np.copy(predictions[0,2])
        real_value = np.copy(labels[0,2])
        pred_value *= 25 # to degree
        real_value *= 25 # to degree
        print("=========")
        print("PRED PITCH: " + str(pred_value))
        print("REAL PITCH: " + str(real_value))

    prediction_copy = np.copy(predictions[:,1])
    labels_copy = np.copy(labels[:,1])
    from sklearn.metrics import mean_squared_error
    RMSE = mean_squared_error(labels_copy, prediction_copy)**0.5
    return RMSE


In [18]:
# Create model
def multilayer_model(_X, _input0, _biases_input0, _hidden1, _biases_hidden1, _hidden2, _biases_hidden2, _output3, _biases_output3):   
    _input0_result = tf.matmul(_X, _input0) + _biases_input0
    _hidden1_result = tf.nn.tanh(tf.matmul(_input0_result, _hidden1) + _biases_hidden1)
    _hidden2_result = tf.nn.tanh(tf.matmul(_hidden1_result, _hidden2) + _biases_hidden2)
    _output3_result = tf.nn.tanh(tf.matmul(_hidden2_result, _output3) + _biases_output3)
    return _output3_result

batch_size = 128 #was 128
graph = tf.Graph()
with graph.as_default():
 
    print("Starting Graph creation...")
    
    # Variables
    image_size = 64
    num_hidden_units_1 = 256 
    num_hidden_units_2 = 256 
    num_hidden_units_3 = 256 
    num_labels = 2
    
    #0- datasets
    tf_train_dataset = tf.placeholder(tf.float32,shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
#     tf_valid_dataset = tf.constant(validation_dataset)
    #Sanitized version
    tf_test_dataset = tf.constant(test_dataset)
    
    #1- weights
    #tf.truncated_normal(shape, mean=0.0, stddev=1.0)
    weights_input0 = tf.Variable(tf.truncated_normal([image_size * image_size, num_hidden_units_1], 0.0, 1.0))    
    weights_hidden1 = tf.Variable(tf.truncated_normal([num_hidden_units_1, num_hidden_units_2], 0.0, 1.0))
    weights_hidden2 = tf.Variable(tf.truncated_normal([num_hidden_units_2, num_hidden_units_3], 0.0, 1.0))
    weights_output3 = tf.Variable(tf.truncated_normal([num_hidden_units_3, num_labels], 0.0, 1.0))    
    
    #2- biases
    biases_input0 = tf.Variable(tf.zeros([num_hidden_units_1]))
    biases_hidden1 = tf.Variable(tf.zeros([num_hidden_units_2]))    
    biases_hidden2 = tf.Variable(tf.zeros([num_hidden_units_3]))
    biases_output3 = tf.Variable(tf.zeros([num_labels]))
 
    #3- Defining a variable for saving the session parameters
    saver = tf.train.Saver({'dnn_weights_input0': weights_input0, 
                            'dnn_weights_hidden1': weights_hidden1,   
                            'dnn_weights_hidden2': weights_hidden2,
                            'dnn_weights_output3': weights_output3,
                            'dnn_biases_input0': biases_input0,
                            'dnn_biases_hidden1': biases_hidden1,
                            'dnn_biases_hidden2': biases_hidden2,
                            'dnn_biases_output3': biases_output3})   
    #4- training
    train_prediction = multilayer_model(tf_train_dataset, 
                     weights_input0, biases_input0, 
                     weights_hidden1, biases_hidden1, 
                     weights_hidden2, biases_hidden2, 
                     weights_output3, biases_output3)
    
    # Minimize the squared errors.
    loss = tf.reduce_mean(tf.square(train_prediction - tf_train_labels))


    #5- Adding the regularization terms to the loss
    beta = 5e-4
    loss += (beta * tf.nn.l2_loss(weights_input0)) 
    loss += (beta * tf.nn.l2_loss(weights_hidden1)) 
    loss += (beta * tf.nn.l2_loss(weights_hidden2)) 
    loss += (beta * tf.nn.l2_loss(weights_output3))
    
    #6- Optimizer.
#     learning_rate = 0.001
    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, 5000, 0.96, staircase=True)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step) 
        
    #Valid prediction
#     valid_prediction = multilayer_model(tf_valid_dataset, 
#                      weights_input0, biases_input0, 
#                      weights_hidden1, biases_hidden1, 
#                      weights_hidden2, biases_hidden2, 
#                      weights_output3, biases_output3)
    #Test prediction
    test_prediction = multilayer_model(tf_test_dataset, 
                     weights_input0, biases_input0, 
                     weights_hidden1, biases_hidden1, 
                     weights_hidden2, biases_hidden2, 
                     weights_output3, biases_output3)
    
    print("Finished.")


Starting Graph creation...
Finished.


In [20]:
num_steps = 50001

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print("Starting Training...")
    print("Train dataset shape: ", training_dataset.shape)
    for step in range(num_steps):
        # Pick an offset within the training data
        offset = (step * batch_size) % (training_label.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = training_dataset[offset:(offset + batch_size), :]
        batch_labels = training_label[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 100 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: " + str(accuracy(predictions, batch_labels)))
#             print("Validation accuracy: " + str(accuracy(valid_prediction.eval(), validation_label, True)))      
  
    saver.save(session, './dnn_1600i_4h_3o', global_step=step) #save the session    
    print("Test accuracy: " + str(accuracy(test_prediction.eval(), test_label)))
    final_test(test_prediction.eval(), test_label)

Starting Training...
Train dataset shape:  (2604, 4096)
Minibatch loss at step 0: 1186.357788
Minibatch accuracy: 35.16198217157808
Minibatch loss at step 100: 1158.364868
Minibatch accuracy: 36.53272358554896
Minibatch loss at step 200: 1238.864868
Minibatch accuracy: 36.526280805101486
Minibatch loss at step 300: 1036.407593
Minibatch accuracy: 33.65081272815896
Minibatch loss at step 400: 1099.081543
Minibatch accuracy: 35.19256457779825
Minibatch loss at step 500: 1130.342529
Minibatch accuracy: 35.63471975409832
Minibatch loss at step 600: 1074.057739
Minibatch accuracy: 33.72118634038733
Minibatch loss at step 700: 1112.431641
Minibatch accuracy: 35.627112396253956
Minibatch loss at step 800: 1173.412964
Minibatch accuracy: 36.79924023714688
Minibatch loss at step 900: 1083.858154
Minibatch accuracy: 36.496351085802054
Minibatch loss at step 1000: 1161.841064
Minibatch accuracy: 37.48235913191364
Minibatch loss at step 1100: 1019.195374
Minibatch accuracy: 33.72218182257785
Minib

Minibatch loss at step 10400: 856.795349
Minibatch accuracy: 32.90800832196853
Minibatch loss at step 10500: 1033.414795
Minibatch accuracy: 38.59258127025922
Minibatch loss at step 10600: 853.577515
Minibatch accuracy: 35.893209800262014
Minibatch loss at step 10700: 1021.399292
Minibatch accuracy: 37.540568875863315
Minibatch loss at step 10800: 989.426086
Minibatch accuracy: 35.834379905380324
Minibatch loss at step 10900: 882.036072
Minibatch accuracy: 34.43633377675932
Minibatch loss at step 11000: 883.674988
Minibatch accuracy: 33.694126928192176
Minibatch loss at step 11100: 1042.419678
Minibatch accuracy: 38.09090111886138
Minibatch loss at step 11200: 850.765442
Minibatch accuracy: 35.500598311419346
Minibatch loss at step 11300: 940.761230
Minibatch accuracy: 36.428688096145464
Minibatch loss at step 11400: 1032.203369
Minibatch accuracy: 37.010694453174615
Minibatch loss at step 11500: 852.919739
Minibatch accuracy: 33.71083701462428
Minibatch loss at step 11600: 952.142273


Minibatch loss at step 20800: 911.375122
Minibatch accuracy: 34.40360670477552
Minibatch loss at step 20900: 786.548828
Minibatch accuracy: 32.9865251253258
Minibatch loss at step 21000: 954.878113
Minibatch accuracy: 35.63230119809658
Minibatch loss at step 21100: 939.831177
Minibatch accuracy: 36.622410184672106
Minibatch loss at step 21200: 887.656677
Minibatch accuracy: 35.56896291778656
Minibatch loss at step 21300: 977.618469
Minibatch accuracy: 36.19876497806935
Minibatch loss at step 21400: 911.162354
Minibatch accuracy: 34.52427991023582
Minibatch loss at step 21500: 807.843567
Minibatch accuracy: 32.6619929908267
Minibatch loss at step 21600: 882.567444
Minibatch accuracy: 33.4251729614074
Minibatch loss at step 21700: 1009.884155
Minibatch accuracy: 37.88189332381468
Minibatch loss at step 21800: 831.993347
Minibatch accuracy: 35.382126808050096
Minibatch loss at step 21900: 1018.797607
Minibatch accuracy: 37.35475942577883
Minibatch loss at step 22000: 990.362549
Minibatch 

Minibatch loss at step 31200: 960.460754
Minibatch accuracy: 34.94970227989253
Minibatch loss at step 31300: 772.446228
Minibatch accuracy: 31.71750882731709
Minibatch loss at step 31400: 982.858643
Minibatch accuracy: 35.541064085479
Minibatch loss at step 31500: 777.185059
Minibatch accuracy: 31.653174594565748
Minibatch loss at step 31600: 961.139160
Minibatch accuracy: 36.32764952645832
Minibatch loss at step 31700: 899.585815
Minibatch accuracy: 35.36204350845338
Minibatch loss at step 31800: 951.946960
Minibatch accuracy: 34.841059313150915
Minibatch loss at step 31900: 922.012512
Minibatch accuracy: 34.41948471047141
Minibatch loss at step 32000: 918.235779
Minibatch accuracy: 34.158580089781246
Minibatch loss at step 32100: 793.365356
Minibatch accuracy: 32.69646148668079
Minibatch loss at step 32200: 974.612427
Minibatch accuracy: 35.98530448213374
Minibatch loss at step 32300: 942.837280
Minibatch accuracy: 36.534968934954975
Minibatch loss at step 32400: 894.241455
Minibatch

Minibatch loss at step 41600: 962.103577
Minibatch accuracy: 36.343612290151825
Minibatch loss at step 41700: 866.653687
Minibatch accuracy: 35.77568529327041
Minibatch loss at step 41800: 901.010132
Minibatch accuracy: 33.042286915547706
Minibatch loss at step 41900: 853.739502
Minibatch accuracy: 34.69605750297863
Minibatch loss at step 42000: 1003.912720
Minibatch accuracy: 36.167227698221495
Minibatch loss at step 42100: 895.559631
Minibatch accuracy: 33.61350981364766
Minibatch loss at step 42200: 971.238403
Minibatch accuracy: 36.11623352758078
Minibatch loss at step 42300: 885.600586
Minibatch accuracy: 35.99593415082633
Minibatch loss at step 42400: 949.960693
Minibatch accuracy: 34.602410063846165
Minibatch loss at step 42500: 784.334900
Minibatch accuracy: 32.187829943697274
Minibatch loss at step 42600: 992.705750
Minibatch accuracy: 35.87927348240206
Minibatch loss at step 42700: 784.862610
Minibatch accuracy: 31.680981208171254
Minibatch loss at step 42800: 965.646973
Mini